In [ ]:
import pandas as pd
import numpy as np
import warnings

from itertools import product
from joblib import Parallel, delayed # for parallelization
from tqdm import tqdm

# Preparation
## Data

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [ ]:
var_list = [
    # Q2
    'clseusa', # 'clsetown', 'clsestat', 'clsenoam',
    # Q3
    'ambornin', 'amcit', 'amlived', 'amenglsh', 
    'amchrstn', 'amgovt', 'amfeel', # 'amancstr',
    # Q4
    'amcitizn', 'amshamed', 'belikeus', 'ambetter', 'ifwrong', # 'amsports', 'lessprd',
    # Q5
    'proudsss', 'proudgrp', 'proudpol', 'prouddem', 'proudeco',
    'proudspt', 'proudart', 'proudhis', 'proudmil', 'proudsci'
]

var_list_f = [var + "_f" for var in var_list]
var_list_n = [var + "_n" for var in var_list]

In [ ]:
# Load imputed data
data2004_i = pd.read_parquet("data/data2004_i.parquet") 

In [ ]:
# Dataset with categorical outcomes
data_f = data2004_i[var_list_f]

## Label encoding
data_f_oh = data_f.apply(lambda col: LabelEncoder().fit_transform(col))

# Dataset with numeric outcomes
data_n = data2004_i[var_list_n]

## Scaling and normalizing
scaler = MinMaxScaler(feature_range=(-1,1))
data_n_scaled = scaler.fit_transform(data_n)

normalizer = StandardScaler()
data_n_norm = normalizer.fit_transform(data_n)

# Dataset with controls
controls = data2004_i[['sex', 'race_f', 'born_usa', 'party_fs', 'religstr_f', 'reltrad_f', 'region_f']]
controls_dum = pd.get_dummies(controls)

## Parameters

In [ ]:
CVI = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
max_clust = 16
max_threads = 8

# 1. Fit models

In [ ]:
from src.model_fit import do_StepMix, do_kmeans, do_AHC, do_hdbscan

## 1.1. Latent

In [ ]:
msrt = ['categorical', 'continuous']
covar = ['without', 'with']
latent_params = list(product(msrt, covar))

clust_range = range(1, max_clust+1)
latent_grid = product(clust_range, latent_params)

In [ ]:
results = Parallel(n_jobs=max_threads)(
    delayed(do_StepMix)(
        data_f_oh if msrt == 'categorical' else data_n,
        controls_dum if covar == 'with' else None,
        n, 
        msrt, 
        covar)
    for n, (msrt, covar) in tqdm(latent_grid, desc='Fitting latent models')
)

latent_all = pd.DataFrame(results)
latent_all.to_csv("output/models/latent_all.csv", index=False)

## 1.2. k-means

In [ ]:
dist = ['euclidean', 'manhattan', 'chebyshev']
link = ['mean', 'median', 'medoid']
kmeans_params = list(product(dist, link))

clust_range = range(2, max_clust+1)
kmeans_grid = product(clust_range, kmeans_params)

In [ ]:
results = Parallel(n_jobs=max_threads)(
    delayed(do_kmeans)(data_n, n, dist, link) 
    for n, (dist, link) in tqdm(kmeans_grid, desc='Fitting KMeans models')
)

kmeans_all = pd.DataFrame(results)

## 1.3. AHC

In [ ]:
distances = ['manhattan', 'euclidean', 'chebyshev', 'hamming']
linkages = ['single', 'average', 'complete']
ahc_params = [*product(distances, linkages), ('euclidean', 'ward')]

clust_range = range(1, max_clust+1)
ahc_grid = product(clust_range, ahc_params)

In [ ]:
results = Parallel(n_jobs=max_threads)(
    delayed(do_AHC)(data_n, n, dist, link) 
    for n, (dist, link) in tqdm(ahc_grid, desc='Fitting AHC models')
)

ahc_all = pd.DataFrame(results)

## 1.4. HDBSCAN

In [ ]:
distances = ['manhattan', 'euclidean', 'chebyshev', 'mahalanobis', 'hamming']
min_cluster_sizes = range(2, 21)
min_samples_range = range(1, 21)
hdb_params = product(distances, min_cluster_sizes, min_samples_range)

In [ ]:
results = Parallel(n_jobs=max_threads)(
    delayed(do_hdbscan)(data_n, dist, min_clust, min_smpl)
    for dist, min_clust, min_smpl in tqdm(hdb_params, desc='Fitting HDBSCAN models')
)

hdbscan_all = pd.DataFrame(results)

## 1.5. Aggregate results and compare CVI

In [ ]:
all_models = pd.concat([latent_all, kmeans_all, ahc_all, hdbscan_all]).reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

all_CVI = all_models[['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']]

correlations = all_CVI.corr(method='spearman')

plt.figure(figsize=(5, 5)) 
sns.heatmap(correlations, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, 
            square=True, linewidths=0.5, vmin=-1, vmax=1)
plt.show()

# 2. Select models

## 2.1. Gap statistics for latent models, kmeans and AHC

In [ ]:
all_models = pd.concat([latent_all, kmeans_all, ahc_all]).reset_index(drop=True)

*Step 1: compute the gap statistic for each model-config*

In [ ]:
# Define parameters grid
models = ['latent', 'kmeans', 'AHC']

params = {
    'latent': latent_params,
    'kmeans': kmeans_params,
    'AHC': ahc_params
}

param_names = {
    'latent': ['msrt', 'covar'],
    'kmeans': ['dist', 'link'],
    'AHC': ['dist', 'link']
}

grid = [
    (model, dict(zip(param_names[model], param_values)))
    for model in models
    for param_values in params[model]
]

In [ ]:
# Compute gap values for all models
from src.model_select import compute_gap_statistic

results = Parallel(n_jobs=max_threads)(
    delayed(compute_gap_statistic)(
        data_f_oh if model == 'latent' and config.get('msrt') == 'categorical' else data_n, 
        all_models, 
        max_clust, 
        CVI, 
        iters=10, 
        model=model, 
        params=config)
    for model, config in tqdm(grid, desc = 'Bootstrapping CVIs')
)

gap_values = pd.concat(results).reset_index(drop=True)

*Step 2: identify the optimal number of clusters for each model-config*

In [ ]:
# Create df to store results
cols = ['model', 'params', 'n_clust'] + \
       [index for index in CVI] + \
       [f'{index}_abs' for index in CVI] + \
       [f'{index}_elbow' for index in CVI] + \
       [f'{index}_gap' for index in CVI]

candidate_models = pd.DataFrame(columns=cols)

candidate_models['model'] = candidate_models['model'].astype('object')
candidate_models['params'] = candidate_models['params'].astype('object')

float_cols = [col for col in cols if col not in ['model', 'params', 'n_clust'] + CVI]
candidate_models[float_cols] = candidate_models[float_cols].astype('float64')

int_cols = [col for col in cols if col in ['n_clust'] + CVI]
candidate_models[int_cols] = candidate_models[int_cols].astype('int64')

In [ ]:
# Find best n
from src.model_select import get_best_gap

for model, config in grid:
    for index in CVI:
        best_n = get_best_gap(gap_values, model, config, index)

        # Check if a best value has been identified
        if best_n != 'none':
            row_id = ((candidate_models['model'] == model) & 
                      (candidate_models['params'] == config) &
                      (candidate_models['n_clust'] == best_n))
            
            # Check if the corresponding row exists in the df
            if candidate_models[row_id].empty:

                model_id = ((all_models['model'] == model) & 
                           (all_models['params'] == config) &
                           (all_models['n_clust'] == best_n))
                
                new_row = {
                    'model': model,
                    'params': config,
                    'n_clust': best_n,
                    'min_clust_size': all_models.loc[model_id, 'min_clust_size'].values[0],
                    'max_clust_size': all_models.loc[model_id, 'max_clust_size'].values[0],
                    'silhouette': all_models.loc[model_id, 'silhouette'].values[0],
                    'calinski_harabasz': all_models.loc[model_id, 'calinski_harabasz'].values[0],
                    'davies_bouldin': all_models.loc[model_id, 'davies_bouldin'].values[0],
                    'dunn': all_models.loc[model_id, 'dunn'].values[0],
                    f'{index}_gap': 1
                }
                
                new_row = pd.DataFrame([new_row])
                candidate_models = pd.concat([candidate_models, new_row], ignore_index=True)

            # Otherwise, update the existing row
            else:
                candidate_models.loc[row_id, f'{index}_gap'] = 1

In [ ]:
candidate_models.to_csv("candidate_models.csv", index=False)

*Step 3: identify the best model for each class*

In [ ]:
CVI_results = {}

for index in CVI:
    CVI_results[index] = []
    df = candidate_models[candidate_models[f'{index}_gap'] == 1]
    
    for model in models:
        sub_df = df[df['model'] == model]

        if sub_df.empty:
            continue
        else:
            if index == 'davies_bouldin':
                best_mod = sub_df.sort_values(index, ascending=True).iloc[0]
            else:
                best_mod = sub_df.sort_values(index, ascending=False).iloc[0]
            CVI_results[index].append(best_mod)

In [ ]:
best_sil = pd.DataFrame(CVI_results['silhouette'])
best_ch = pd.DataFrame(CVI_results['calinski_harabasz'])
best_db = pd.DataFrame(CVI_results['davies_bouldin'])
best_dunn = pd.DataFrame(CVI_results['dunn'])

## 2.2. Min/max for HDBSCAN

In [ ]:
best_sil = pd.concat([best_sil, hdbscan_all.sort_values('silhouette', ascending=False).iloc[0:1]], axis=0)
best_ch = pd.concat([best_ch, hdbscan_all.sort_values('calinski_harabasz', ascending=False).iloc[0:1]], axis=0)
best_db = pd.concat([best_db, hdbscan_all.sort_values('davies_bouldin', ascending=True).iloc[0:1]], axis=0)
best_dunn = pd.concat([best_dunn, hdbscan_all.sort_values('dunn', ascending=False).iloc[0:1]], axis=0)

In [ ]:
best_sil = best_sil.drop(columns=[col for col in best_sil.columns if col.endswith(('elbow', 'abs', 'gap'))])
best_ch = best_ch.drop(columns=[col for col in best_ch.columns if col.endswith(('elbow', 'abs', 'gap'))])
best_db = best_db.drop(columns=[col for col in best_db.columns if col.endswith(('elbow', 'abs', 'gap'))])
best_dunn = best_dunn.drop(columns=[col for col in best_dunn.columns if col.endswith(('elbow', 'abs', 'gap'))])

## 2.3. For latent models...

In [ ]:
latent_stats = latent_all.drop(columns=['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn'])

In [ ]:
def disp_params(d):
    return f"{d['msrt']} {d['covar']} covariates"

### 2.3.1. Absolute value for AIC / BIC / entropy

In [ ]:
best_aic = latent_all.sort_values('aic', ascending=True).iloc[0]
best_bic = latent_all.sort_values('bic', ascending=True).iloc[0]
best_entropy = latent_all.sort_values('entropy', ascending=False).iloc[0]

print(f"Model minimizing AIC is {disp_params(best_aic['params'])} with {best_aic['n_clust']} clusters.")
print(f"Model minimizing BIC is {disp_params(best_bic['params'])} with {best_bic['n_clust']} clusters.")
print(f"Model maximizing entropy is {disp_params(best_entropy['params'])} with {best_entropy['n_clust']} clusters.")

### 2.3.2. Elbow method for AIC / BIC / entropy

In [ ]:
from kneed import KneeLocator

def elbow_method(df, val_index):
    res = df.dropna(subset=[val_index])

    x = res["n_clust"]
    y = res[val_index]

    if val_index == 'entropy':
        knee_locator = KneeLocator(x, y, curve='concave', direction='increasing')
    else:
        knee_locator = KneeLocator(x, y, curve='convex', direction='decreasing')
    
    return res[res["n_clust"] == knee_locator.knee]

def best_elbow_model(index):
    candidate_models = pd.DataFrame()

    for msrt in ['categorical', 'continuous']:
        for covar in ['without']:
            # for covar in ['without', 'with']:
            mask = (latent_stats['params'] == {'msrt': msrt, 'covar': covar})
            models = latent_stats[mask]
            elbow_res = elbow_method(models, index)
            candidate_models = pd.concat([candidate_models, elbow_res], ignore_index=True)

    best_model = candidate_models.sort_values(index, ascending=True).iloc[0]
    return pd.DataFrame(best_model).T

In [ ]:
best_aic = best_elbow_model('aic')
best_bic = best_elbow_model('bic')
best_entropy = best_elbow_model('entropy')

print(f"Best model according to the Elbow method applied to...")
print(f"- AIC is {best_aic['params'].iloc[0]} with {best_aic['n_clust'].iloc[0]} clusters.")
print(f"- BIC is {best_bic['params'].iloc[0]} with {best_bic['n_clust'].iloc[0]} clusters.")
print(f"- Entropy is {best_entropy['params'].iloc[0]} with {best_entropy['n_clust'].iloc[0]} clusters.")

### 2.3.3. Statistical tests
*LRT*

In [ ]:
from scipy.stats import chi2

def LRT(models):
    l2_red = (models['LL'].iloc[0] - models['LL']) / models['LL'].iloc[0]
    lik_rat = 2 * (models['LL'] - models['LL'].iloc[0])
    d_df = models['df'] - models['df'].iloc[0]
    p_val = 1 - chi2.cdf(lik_rat, d_df)

    results = pd.DataFrame({
        'L2 reduction': l2_red,
        'LR ratio': lik_rat,
        'LR pval': p_val
    }, index=models.index)
    
    return results

In [ ]:
for msrt in ['categorical', 'continuous']:
    for covar in ['without', 'with']:
        mask = (latent_stats['params'] == {'msrt': msrt, 'covar': covar})
        models = latent_stats[mask]
        lrt_results = LRT(models)
        latent_stats.loc[mask, ['L2 reduction', 'LR ratio', 'LR pval']] = lrt_results.values

*BLRT - not implemented in StepMix for models with covariates.*

In [ ]:
from stepmix.stepmix import StepMix
from stepmix.bootstrap import blrt_sweep

opt_params = {
    'method': 'gradient',
    'intercept': True,
    'max_iter': 2500,
}

for msrt in ['categorical', 'continuous']:
    latent_mod = StepMix(
        measurement = msrt,
        n_init = 3,
        init_params = 'kmeans',
        structural_params = opt_params,
        progress_bar = 0)
 
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)

        blrt = blrt_sweep(
            latent_mod,
            data_f_oh if msrt == 'categorical' else data_n,
            low=1,
            high=max_clust,
            n_repetitions=20)
            
        blrt = pd.concat([pd.DataFrame({'p': [np.nan]}), blrt]).reset_index(drop=True) # Add a row for the saturated model

    mask = (latent_stats['params'] == {'msrt': msrt, 'covar': 'without'})
    latent_stats.loc[mask, ['BLR pval']] = blrt.values

In [ ]:
LCA = latent_stats[latent_stats['params'] == {'msrt': 'categorical', 'covar': 'without'}].reset_index(drop=True)
LPA = latent_stats[latent_stats['params'] == {'msrt': 'continuous', 'covar': 'without'}].reset_index(drop=True)

LCA.to_csv("bootstrapped_LCA.csv", index=False)
LPA.to_csv("bootstrapped_LPA.csv", index=False)

*Results*

In [ ]:
best_LCA = LCA[LCA['BLRT'] > 0.05].iloc[0]['n_clust']
best_LPA = LPA[LPA['BLRT'] > 0.05].iloc[0]['n_clust']

print(f"Optimal number of clusters for LCA according to BLRT is {best_LCA}.")
print(f"Optimal number of clusters for LPA according to BLRT is {best_LPA}.")

In [ ]:
# LCA.drop(columns=['model', 'params', 'aic', 'bic', 'entropy'])

In [ ]:
# LPA.drop(columns=['model', 'params', 'aic', 'bic', 'entropy'])

# 3. Results

In [ ]:
from src.model_plot import plot_clusters, plot_cluster_profiles

In [ ]:
def refit_best_model(df):
    model = df.loc[0, 'model']
    config = df.loc[0, 'params']
    n_clust = int(df.loc[0, 'n_clust'])

    if model == 'latent':
        results = do_StepMix(data_f_oh if config['msrt'] == 'categorical' else data_n, n_clust, refit = True, **config)
    
    elif model == 'kmeans':
        results = do_kmeans(data_n, n_clust, refit = True, **config)
    
    elif model == 'AHC':
        results = do_AHC(data_n, n_clust, refit = True, **config)
    
    elif model == 'HDBSCAN':
        results = do_hdbscan(data_n, refit = True, **config)

    return results

## 3.1. Silhouette

In [ ]:
sil = best_sil.sort_values('silhouette', ascending=False).drop(columns=['calinski_harabasz', 'davies_bouldin', 'dunn']).reset_index(drop=True)
sil

In [ ]:
pred_clust_sil = refit_best_model(sil)

In [ ]:
plot_clusters(
    data_f_oh if (sil.loc[0, 'model'] == 'latent') and (sil.loc[0, 'params'].get('msrt') == 'categorical') else data_n,
    pred_clust_sil,
    '2D PCA Projection of the Best Partition According to the Silhouette Index')

In [ ]:
plot_cluster_profiles(data_n, pred_clust_sil, feature_names = var_list, sd = 1, title = 'Silhouette')

## 3.2. Calinski-Harabasz

In [ ]:
ch = best_ch.sort_values('calinski_harabasz', ascending=False).drop(columns=['silhouette', 'davies_bouldin', 'dunn']).reset_index(drop=True)
ch

In [ ]:
pred_clust_ch = refit_best_model(ch)

In [ ]:
plot_clusters(
    data_f_oh if (ch.loc[0, 'model'] == 'latent') and (ch.loc[0, 'params'].get('msrt') == 'categorical') else data_n, 
    pred_clust_ch,
    '2D PCA Projection of the Best Partition According to the Calinski-Harabasz Index')

In [ ]:
plot_cluster_profiles(data_n, pred_clust_ch, feature_names = var_list, sd = 1, title = 'Calinski-Harabaz')

## 3.3. Davies-Bouldin

In [ ]:
db = best_db.sort_values('davies_bouldin', ascending=True).drop(columns=['silhouette', 'calinski_harabasz', 'dunn']).reset_index(drop=True)
db

In [ ]:
pred_clust_db = refit_best_model(db)

In [ ]:
plot_clusters(
    data_f_oh if (db.loc[0, 'model'] == 'latent') and (db.loc[0, 'params'].get('msrt') == 'categorical') else data_n, 
    pred_clust_db,
    '2D PCA Projection of the Best Partition According to the Davies-Bouldin Index')

In [ ]:
plot_cluster_profiles(data_n, pred_clust_db, feature_names = var_list, sd = 1, title = 'Davies-Bouldin')

## 3.4. Generalized Dunn 43

In [ ]:
gd = best_dunn.sort_values('dunn', ascending=False).drop(columns=['silhouette', 'calinski_harabasz', 'davies_bouldin']).reset_index(drop=True)
gd

In [ ]:
pred_clust_gd = refit_best_model(gd)

In [ ]:
plot_clusters(
    data_f_oh if (gd.loc[0, 'model'] == 'latent') and (gd.loc[0, 'params'].get('msrt') == 'categorical') else data_n, 
    pred_clust_gd,
    '2D PCA Projection of the Best Partition According to the Generalized Dunn Index')

In [ ]:
plot_cluster_profiles(data_n, pred_clust_gd, feature_names = var_list, sd = 1, title = 'Generalized Dunn')

# 4. Clusterability - Hopkins Statistic

Function from the pyclustertend package, which could not be installed because its depencies are outdated.
See: https://pyclustertend.readthedocs.io/en/latest/_modules/pyclustertend/hopkins.html

In [ ]:
from src.hopkins import hopkins

In [ ]:
float(hopkins(data_n.values, data_n.shape[0]))